# Fundamentos de Machine Learning Supervisado

**Curso:** “Fundamentos de Programación y Analítica de Datos con Python”  
**Duración estimada del bloque:** 2 horas

## Objetivos específicos
- Diferenciar formalmente entre aprendizaje supervisado, clasificación y regresión, identificando variables de entrada y variable objetivo.
- Preparar datos básicos para clasificación (train/test split, estandarización opcional) y justificar decisiones.
- Entrenar y evaluar modelos de clasificación binaria y multiclase con regresión logística y árboles de decisión.
- Seleccionar y reportar métricas adecuadas (accuracy, precision, recall, F1, AUC-ROC) según el contexto del problema.

## Prerrequisitos
- Conocimientos fundamentales de Python (tipos, control de flujo, funciones).
- Nociones de NumPy, Pandas y visualización básica con Matplotlib.
- Instalación de librerías: `numpy`, `pandas`, `scikit-learn`, `matplotlib`.

## Tema 1 — Introducción al Aprendizaje Supervisado

### Definición
El aprendizaje supervisado es un paradigma de *Machine Learning* en el que se dispone de **observaciones etiquetadas** \((\mathbf{x}_i, y_i)\), donde \(\mathbf{x}_i\) es un vector de características y \(y_i\) es la etiqueta (clase para clasificación; valor continuo para regresión). El objetivo es **aprender una función** \(f: \mathbb{R}^p \to \mathcal{Y}\) que generalice y permita predecir \(y\) para **nuevas** instancias \(\mathbf{x}\).

### Importancia en programación y analítica de datos
- Permite construir **sistemas predictivos** para soporte de decisiones (por ejemplo, detección de fraude o clasificación de spam).
- Exige una **pipeline reproducible**: carga de datos, preprocesamiento, entrenamiento, evaluación y despliegue.
- Introduce buenas prácticas de **validación** (train/test split, cross-validation) y **métricas** pertinentes al negocio.

### Buenas prácticas profesionales y errores comunes
- **Buenas prácticas:**
  - Fijar `random_state` para resultados reproducibles.
  - Separar conjunto de **entrenamiento** y **prueba** antes de explorar desempeño.
  - Estandarizar características cuando el modelo sea **sensible a escala** (e.g., modelos lineales).
- **Errores comunes:**
  - Fuga de datos (*data leakage*): usar información del conjunto de prueba durante el preprocesamiento.
  - Confiar solo en `accuracy` en conjuntos desbalanceados; preferir F1, AUC-ROC o precisión/recobrado por clase.

### Ejemplo en Python
A continuación, se ilustra el flujo mínimo: carga de un dataset de `scikit-learn`, división train/test y verificación de tamaños.

In [ ]:
# Flujo mínimo de aprendizaje supervisado: carga de datos y train/test split
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import numpy as np

# 1) Cargar dataset (clasificación binaria)
X, y = load_breast_cancer(return_X_y=True)

# 2) División estratificada para preservar proporciones de clases
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, stratify=y, random_state=42
)

print("Dimensiones X:", X.shape, "| y:", y.shape)
print("Train:", X_train.shape, y_train.shape, "| Test:", X_test.shape, y_test.shape)

# Distribución de clases
unique, counts = np.unique(y, return_counts=True)
print("Distribución total de clases:", dict(zip(unique, counts)))


## Tema 2 — Regresión Logística (Clasificación Binaria)

### Definición
La **regresión logística** modela la **probabilidad** de pertenecer a una clase (por ejemplo, \(y=1\)) mediante la función sigmoide:  
\[ \hat{p}(y=1\mid \mathbf{x}) = \sigma(\mathbf{w}^\top \mathbf{x} + b), \quad \sigma(z) = \frac{1}{1 + e^{-z}}. \]
Se entrena minimizando la **entropía cruzada** (log-loss). La decisión de clase se obtiene con un **umbral** (por defecto 0.5).

### Importancia en programación y analítica de datos
- Modelo **rápido** y base para líneas de producción con alta interpretabilidad (coeficientes).
- Adecuado como **baseline** en numerosos problemas antes de modelos más complejos.
- Requiere atención a **escala** de variables y **regularización** para evitar sobreajuste.

### Buenas prácticas profesionales y errores comunes
- Escalar características (e.g., `StandardScaler`) para mejorar la convergencia.
- Verificar **multicolinealidad** y realizar selección/ingeniería de características cuando sea necesario.
- Evitar interpretar coeficientes sin considerar escala y regularización.

### Ejemplo en Python
Entrenamiento, predicción y métricas básicas en un dataset binario.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, RocCurveDisplay
import matplotlib.pyplot as plt

# Pipeline: estandarización + regresión logística
logreg = make_pipeline(
    StandardScaler(),
    LogisticRegression(max_iter=1000, random_state=42)
)

logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
y_proba = logreg.predict_proba(X_test)[:, 1]

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# AUC-ROC
auc = roc_auc_score(y_test, y_proba)
print("AUC-ROC:", round(auc, 4))

# Curva ROC
RocCurveDisplay.from_estimator(logreg, X_test, y_test)
plt.title("Curva ROC - Regresión Logística")
plt.show()


## Tema 3 — Árboles de Decisión

### Definición
Un **árbol de decisión** realiza particiones recursivas del espacio de características usando reglas del tipo *si-entonces*. En clasificación, utiliza medidas de impureza como **Gini** o **Entropía** para seleccionar divisiones.

### Importancia en programación y analítica de datos
- Modelo **interpretable** (reglas) y capaz de capturar **relaciones no lineales** e **interacciones**.
- Suele requerir **menos preprocesamiento** que modelos lineales.
- Constituye la base de métodos **ensamblados** (Random Forest, Gradient Boosting).

### Buenas prácticas profesionales y errores comunes
- Controlar la **profundidad** (`max_depth`) y el **número mínimo de muestras por división** para evitar sobreajuste.
- Evaluar desempeño con **validación cruzada** si es posible.
- No utilizar árboles sin restricción en datasets pequeños: alta varianza.

### Ejemplo en Python
Entrenamiento, visualización sintética del árbol y evaluación de métricas.

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt

tree_clf = DecisionTreeClassifier(
    criterion="gini",
    max_depth=4,
    random_state=42
)

tree_clf.fit(X_train, y_train)
y_pred_tree = tree_clf.predict(X_test)

# Métricas básicas
from sklearn.metrics import accuracy_score, classification_report
print("Accuracy (Árbol):", accuracy_score(y_test, y_pred_tree))
print("\nClassification Report (Árbol):\n", classification_report(y_test, y_pred_tree))

# Matriz de confusión
ConfusionMatrixDisplay.from_estimator(tree_clf, X_test, y_test)
plt.title("Matriz de Confusión - Árbol de Decisión")
plt.show()

# Visualización del árbol (puede ser grande; se fija max_depth para claridad)
plt.figure(figsize=(14, 6))
plot_tree(
    tree_clf,
    filled=True,
    feature_names=load_breast_cancer().feature_names,
    class_names=["No", "Sí"]
)
plt.title("Árbol de Decisión (profundidad limitada)")
plt.show()


## Tema 4 — Evaluación de Modelos Supervisados

### Definición
La **evaluación** valora la calidad predictiva de un modelo en datos **no vistos**. En clasificación se emplean métricas como:
- **Accuracy**: proporción de aciertos totales.
- **Precision/Recall**: precisión y exhaustividad para clases positivas.
- **F1-score**: media armónica de precisión y recall.
- **AUC-ROC**: área bajo la curva ROC, robusta a umbrales.

### Importancia en programación y analítica de datos
- Permite **comparar modelos** y seleccionar configuraciones de hiperparámetros.
- Fomenta decisiones informadas según el **costo del error** (falsos positivos vs falsos negativos).
- Esencial para comunicar resultados a **stakeholders**.

### Buenas prácticas profesionales y errores comunes
- Reportar **múltiples métricas** y **curvas** (ROC, PR) cuando hay desbalance.
- Usar **estratificación** en `train_test_split` y **validación cruzada** para estimar generalización.
- Evitar optimizar exclusivamente una métrica sin comprender el contexto del problema.

### Ejemplo en Python
Comparación rápida entre modelos (regresión logística vs árbol) usando F1 macro y AUC cuando aplique.

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression

# Aseguramos dos modelos comparables
logreg_model = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000, random_state=42))
tree_model = DecisionTreeClassifier(max_depth=4, random_state=42)

logreg_model.fit(X_train, y_train)
tree_model.fit(X_train, y_train)

# Predicciones
y_pred_log = logreg_model.predict(X_test)
y_proba_log = logreg_model.predict_proba(X_test)[:, 1]
y_pred_tree = tree_model.predict(X_test)

# Métricas
f1_log = f1_score(y_test, y_pred_log, average="macro")
f1_tree = f1_score(y_test, y_pred_tree, average="macro")
auc_log = roc_auc_score(y_test, y_proba_log)

print(f"F1 (LogReg): {f1_log:.4f}")
print(f"F1 (Árbol):  {f1_tree:.4f}")
print(f"AUC (LogReg): {auc_log:.4f}  # AUC no aplica directamente a predicciones discretas del árbol sin calibrated proba")


# Ejercicios Integradores

A continuación, se proponen ejercicios que articulan los conceptos del bloque. Cada ejercicio incluye **contexto técnico**, **datos/entradas**, **requerimientos**, **criterios de aceptación** y **pistas**.

## Ejercicio 1 — Baseline reproducible con Regresión Logística

**Contexto técnico:** Eres analista de datos en una clínica y necesitas un **baseline** reproducible para clasificar pacientes en dos grupos según medidas diagnósticas. El equipo clínico solicita un primer **informe de desempeño**.

**Datos/entradas:** Usa `load_breast_cancer` de `sklearn`. Divide los datos en `train/test` con estratificación y `random_state=42`.

**Requerimientos:**
- Construir un *pipeline* `StandardScaler` + `LogisticRegression(max_iter=1000)`.
- Reportar `accuracy`, `precision`, `recall`, `F1` y `AUC-ROC` en test.
- Graficar la curva ROC.

**Criterios de aceptación:**
- Código reproducible y bien comentado.
- Reporte impreso de métricas y visualización de la curva ROC.
- Explicación breve de cuándo **accuracy** puede ser insuficiente.

**Pistas:**
- Usa `make_pipeline`, `classification_report`, `RocCurveDisplay`.

In [ ]:
# Solución propuesta (puedes ocultar esta celda al evaluar)
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, RocCurveDisplay
import matplotlib.pyplot as plt

X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, stratify=y, random_state=42
)

pipe = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000, random_state=42))
pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)
y_proba = pipe.predict_proba(X_test)[:, 1]

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("AUC-ROC:", roc_auc_score(y_test, y_proba))

RocCurveDisplay.from_estimator(pipe, X_test, y_test)
plt.title("Curva ROC - Baseline Regresión Logística")
plt.show()


## Ejercicio 2 — Árbol de Decisión con control de complejidad

**Contexto técnico:** En una auditoría de calidad, necesitas un modelo interpretable con **reglas** claras para justificar decisiones ante un auditor externo.

**Datos/entradas:** Usa `load_breast_cancer`. División 75/25 con estratificación, `random_state=7`.

**Requerimientos:**
- Entrenar `DecisionTreeClassifier` con `max_depth` en {3, 5, None} y compararlos.
- Reportar `accuracy` y `F1 macro` en test para cada profundidad.
- Graficar la **matriz de confusión** del mejor modelo.

**Criterios de aceptación:**
- Evidencia de **sobreajuste** cuando `max_depth=None` frente a profundidades limitadas.
- Selección justificada del mejor modelo con base en métricas.

**Pistas:**
- Usa `ConfusionMatrixDisplay` y asegúrate de fijar `random_state`.

In [ ]:
# Solución propuesta
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, accuracy_score, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, stratify=y, random_state=7
)

best_model, best_f1 = None, -1
for depth in [3, 5, None]:
    clf = DecisionTreeClassifier(max_depth=depth, random_state=7)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    f1 = f1_score(y_test, y_pred, average="macro")
    acc = accuracy_score(y_test, y_pred)
    print(f"Depth={depth} -> F1={f1:.4f} | Acc={acc:.4f}")
    if f1 > best_f1:
        best_f1, best_model = f1, clf

print("\nMejor profundidad según F1:", best_model.get_params().get("max_depth"))
ConfusionMatrixDisplay.from_estimator(best_model, X_test, y_test)
plt.title("Matriz de confusión - Mejor Árbol")
plt.show()


## Ejercicio 3 — Elección de métrica bajo desbalance

**Contexto técnico:** En un sistema de monitoreo, los **falsos negativos** son costosos. Se te pide priorizar **recall** sobre otras métricas.

**Datos/entradas:** Crea un dataset sintético desbalanceado con `make_classification` (clase positiva 10–15%).

**Requerimientos:**
- Entrenar Regresión Logística y Árbol (con `class_weight='balanced'` en ambos cuando aplique).
- Comparar `recall`, `precision` y `F1` de ambos modelos en test.
- Ajustar el **umbral** de decisión de la logística para maximizar `recall` manteniendo `precision` aceptable.

**Criterios de aceptación:**
- Mostrar cómo `class_weight` y el **umbral** afectan las métricas.
- Reporte claro de la **trade-off** precisión–recobrado.

**Pistas:**
- Usa `predict_proba` en la logística y barre umbrales con `np.linspace`.

In [ ]:
# Solución propuesta
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_score, recall_score, f1_score

# Dataset desbalanceado
X, y = make_classification(
    n_samples=3000, n_features=12, n_informative=6, n_redundant=2,
    weights=[0.88, 0.12], flip_y=0.01, random_state=42
)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, stratify=y, random_state=42
)

# Modelos con class_weight='balanced'
logreg = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000, class_weight="balanced", random_state=42))
tree = DecisionTreeClassifier(max_depth=6, class_weight="balanced", random_state=42)

logreg.fit(X_train, y_train)
tree.fit(X_train, y_train)

# Predicciones con umbral por defecto
y_pred_log = logreg.predict(X_test)
y_pred_tree = tree.predict(X_test)

def report(name, y_true, y_pred):
    p = precision_score(y_true, y_pred, zero_division=0)
    r = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    print(f"{name:>10s} -> Precision={p:.3f} | Recall={r:.3f} | F1={f1:.3f}")

report("LogReg@0.5", y_test, y_pred_log)
report("Tree", y_test, y_pred_tree)

# Barrido de umbral para maximizar recall aceptable
y_proba = logreg.predict_proba(X_test)[:, 1]
thresholds = np.linspace(0.05, 0.8, 16)
best = None
for t in thresholds:
    y_hat = (y_proba >= t).astype(int)
    p = precision_score(y_test, y_hat, zero_division=0)
    r = recall_score(y_test, y_hat)
    f1 = f1_score(y_test, y_hat)
    if best is None or (r > best[1] and p >= 0.30):  # ejemplo de restricción mínima de precisión
        best = (t, r, p, f1)

print(f"\nMejor umbral con precisión >= 0.30 -> t={best[0]:.2f}, recall={best[1]:.3f}, precision={best[2]:.3f}, F1={best[3]:.3f}")


## Ejercicio 4 — Validación cruzada y selección simple de modelo

**Contexto técnico:** Como *ML engineer* debes presentar una **comparación justa** entre dos modelos base usando validación cruzada estratificada.

**Datos/Eentradas:** Usa `load_breast_cancer`. Emplea `StratifiedKFold(n_splits=5, shuffle=True, random_state=42)`.

**Requerimientos:**
- Comparar `LogisticRegression` (con `StandardScaler`) vs `DecisionTreeClassifier` (profundidad 3–6).
- Reportar media y desviación estándar de `F1 macro` por validación cruzada.
- Seleccionar el mejor y reentrenar en el conjunto completo de entrenamiento, evaluando en test.

**Criterios de aceptación:**
- Uso correcto de `Pipeline`/`make_pipeline` y `cross_val_score`.
- Justificación de la elección final basada en **estadística descriptiva** de CV.

**Pistas:**
- Usa `cross_val_score` con `scoring='f1_macro'`.

In [ ]:
# Solución propuesta
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
import numpy as np

X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, stratify=y, random_state=42
)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

logreg = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000, random_state=42))
tree_candidates = [DecisionTreeClassifier(max_depth=d, random_state=42) for d in range(3, 7)]

def cv_summary(model):
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring="f1_macro")
    return np.mean(scores), np.std(scores)

mean_log, std_log = cv_summary(logreg)
print(f"LogReg  -> F1_macro CV: {mean_log:.4f} ± {std_log:.4f}")

best_tree, best_mean, best_std = None, -1, None
for m in tree_candidates:
    mean, std = cv_summary(m)
    print(f"Tree(d={m.max_depth}) -> F1_macro CV: {mean:.4f} ± {std:.4f}")
    if mean > best_mean:
        best_tree, best_mean, best_std = m, mean, std

# Selección
print(f"\nMejor árbol en CV: depth={best_tree.max_depth} con {best_mean:.4f} ± {best_std:.4f}")

# Comparación final reentrenando y evaluando en test
logreg.fit(X_train, y_train)
best_tree.fit(X_train, y_train)

y_pred_log = logreg.predict(X_test)
y_pred_tree = best_tree.predict(X_test)

print(f"F1_macro Test (LogReg): {f1_score(y_test, y_pred_log, average='macro'):.4f}")
print(f"F1_macro Test (Tree):   {f1_score(y_test, y_pred_tree, average='macro'):.4f}")
